In [1]:
import os

from collections import defaultdict
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str='cl100k_base') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

from dotenv import load_dotenv

load_dotenv()
# os.environ["OPENAI_API_KEY"] = ""

True

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader, UnstructuredExcelLoader
from langchain.document_loaders import DirectoryLoader

In [3]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
# loader = DirectoryLoader('./data/', glob="./*.pdf", loader_cls=PyMuPDFLoader)
file_path='data/서울특별시 버스노선 혼잡도 예측을 통한 다람쥐버스 신규 노선제안(장려).pdf'
loader = PyMuPDFLoader(file_path=file_path)
documents = loader.load()

In [4]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

## QA DB

In [15]:
embedding = OpenAIEmbeddings()
persist_directory = 'doc_db'

doc_vectordb = Chroma.from_documents(documents=docs, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [16]:
doc_vectordb.persist()
doc_vectordb = None

In [17]:
doc_vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

In [18]:
retriever = doc_vectordb.as_retriever(search_kwargs={"k": 2})

In [19]:
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

## 표절검사

In [127]:
persist_directory = 'total_db'

total_vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

In [30]:
doc_vectordb.get(include=['embeddings']).keys()

dict_keys(['ids', 'embeddings', 'documents', 'metadatas'])

In [44]:
from tqdm import tqdm

In [66]:
getVectorDBInfo.keys()

dict_keys(['ids', 'embeddings', 'documents', 'metadatas'])

In [54]:
getVectorDBInfo = doc_vectordb.get(include=['embeddings'])
# for ids, vector in zip(getVectorDBInfo['ids'], getVectorDBInfo['embeddings']):
for ids, vector in tqdm(zip(getVectorDBInfo['ids'], getVectorDBInfo['embeddings'])):
    print(ids)

24it [00:00, 240821.28it/s]

f676a69e-1d59-11ee-be65-56cc850cf3bf
f676a6f8-1d59-11ee-be65-56cc850cf3bf
f676a720-1d59-11ee-be65-56cc850cf3bf
f676a748-1d59-11ee-be65-56cc850cf3bf
f676a766-1d59-11ee-be65-56cc850cf3bf
f676a784-1d59-11ee-be65-56cc850cf3bf
f676a7a2-1d59-11ee-be65-56cc850cf3bf
f676a7c0-1d59-11ee-be65-56cc850cf3bf
5aa40a3c-22f8-11ee-8525-56cc850cf3bf
5aa40aa0-22f8-11ee-8525-56cc850cf3bf
5aa40ab4-22f8-11ee-8525-56cc850cf3bf
5aa40ac8-22f8-11ee-8525-56cc850cf3bf
5aa40adc-22f8-11ee-8525-56cc850cf3bf
5aa40af0-22f8-11ee-8525-56cc850cf3bf
5aa40b04-22f8-11ee-8525-56cc850cf3bf
5aa40b18-22f8-11ee-8525-56cc850cf3bf
ad193cec-22f8-11ee-8525-56cc850cf3bf
ad193d82-22f8-11ee-8525-56cc850cf3bf
ad193daa-22f8-11ee-8525-56cc850cf3bf
ad193dc8-22f8-11ee-8525-56cc850cf3bf
ad193de6-22f8-11ee-8525-56cc850cf3bf
ad193e04-22f8-11ee-8525-56cc850cf3bf
ad193e22-22f8-11ee-8525-56cc850cf3bf
ad193e40-22f8-11ee-8525-56cc850cf3bf


In [83]:
def sim(query):
    return doc_vectordb.similarity_search_with_score(query=query, distance_metric="cos", k = 4, include=[ids])

In [84]:
k = sim('query')
k

[(Document(page_content='서울특별시 버스 노선 혼잡도 예측을 통한 다람쥐버스 신규 노선 \n제안\n1. 주제 선정 및 자료분석 배경 (주제 선정의 독창성) \n을지로, 여의도 같은 기업의 밀집도가 높은 지역들의 출퇴근 시간에는 대중교통, 자가용 할 것 없이 모\n든 이동수단의 포화상태를 볼 수 있다. 출퇴근 시간에서의 교통 혼잡도를 줄이기 위해서 대중교통의 이\n용을 적극 홍보하고 있지만, 사실상 버스와 지하철을 타지 못하고 몇 대씩 보내고 난 다음에야 탑승할 \n수 있을 정도로 이용객이 넘쳐난다. 이러한 문제를 해결하기 위해 서울시에서 다람쥐버스의 운행을 시\n작하였다. 다람쥐버스는 서울시에서 운행하는 출퇴근 맞춤버스로 출퇴근 시간대의 특정 구간에서 발생\n하는 차내 혼잡을 해소하기 위해 운행하는 순환형 셔틀버스이다. 다람쥐버스라는 이름은 다람쥐가 쳇바\n퀴 돌듯 짧은 구간을 반복 운행한다고 하여 붙여졌다.\n2017년 6월에 4개의 노선으로 반년 간의 시범 운행을 한 결과 차내 혼잡을 39.5% 완화하는 효과와 동\n시에 해당 구간의 전체 버스 이용객이 16.5% 증가하는 효과를 볼 수 있었다. 이러한 결과를 반영하여 \n2018년에는 3개의 노선이 추가되어 현재는 총 7개의 노선이 운행 중이다.\n다람쥐버스는 현재 이미 시행 중인 정책이지만 2018년에 국토교통부가 주관한 2018 지속가능 교통도시\n평가에서 최우수정책으로 선정될 만큼 긍정적인 평가를 받고 있고 타 지역에서의 운행을 기대하는 시민\n들의 목소리도 높은 것으로 나타났다. 따라서 이번 과제에서 버스 정류소 및 노선별 이용객 수를 예측\n하고 버스 내 혼잡도를 계산함으로써 가장 큰 효과를 얻을 수 있는 신규 노선을 제안하고자 한다. \n2. 분석 내용 (자료분석의 우수성, 데이터 활용성)\n활용데이터\n『서울시 버스노선별, 정류장별, 시간대별 승·하차 인원 정보』 데이터에서 일별, 시간대별 승·하차 인원을 \n구하기 위해 출근 시간대인  7시~9시의 승·하차 인원을 추출, 

In [70]:
resList = k

In [81]:
resList

[(Document(page_content='서울특별시 버스 노선 혼잡도 예측을 통한 다람쥐버스 신규 노선 \n제안\n1. 주제 선정 및 자료분석 배경 (주제 선정의 독창성) \n을지로, 여의도 같은 기업의 밀집도가 높은 지역들의 출퇴근 시간에는 대중교통, 자가용 할 것 없이 모\n든 이동수단의 포화상태를 볼 수 있다. 출퇴근 시간에서의 교통 혼잡도를 줄이기 위해서 대중교통의 이\n용을 적극 홍보하고 있지만, 사실상 버스와 지하철을 타지 못하고 몇 대씩 보내고 난 다음에야 탑승할 \n수 있을 정도로 이용객이 넘쳐난다. 이러한 문제를 해결하기 위해 서울시에서 다람쥐버스의 운행을 시\n작하였다. 다람쥐버스는 서울시에서 운행하는 출퇴근 맞춤버스로 출퇴근 시간대의 특정 구간에서 발생\n하는 차내 혼잡을 해소하기 위해 운행하는 순환형 셔틀버스이다. 다람쥐버스라는 이름은 다람쥐가 쳇바\n퀴 돌듯 짧은 구간을 반복 운행한다고 하여 붙여졌다.\n2017년 6월에 4개의 노선으로 반년 간의 시범 운행을 한 결과 차내 혼잡을 39.5% 완화하는 효과와 동\n시에 해당 구간의 전체 버스 이용객이 16.5% 증가하는 효과를 볼 수 있었다. 이러한 결과를 반영하여 \n2018년에는 3개의 노선이 추가되어 현재는 총 7개의 노선이 운행 중이다.\n다람쥐버스는 현재 이미 시행 중인 정책이지만 2018년에 국토교통부가 주관한 2018 지속가능 교통도시\n평가에서 최우수정책으로 선정될 만큼 긍정적인 평가를 받고 있고 타 지역에서의 운행을 기대하는 시민\n들의 목소리도 높은 것으로 나타났다. 따라서 이번 과제에서 버스 정류소 및 노선별 이용객 수를 예측\n하고 버스 내 혼잡도를 계산함으로써 가장 큰 효과를 얻을 수 있는 신규 노선을 제안하고자 한다. \n2. 분석 내용 (자료분석의 우수성, 데이터 활용성)\n활용데이터\n『서울시 버스노선별, 정류장별, 시간대별 승·하차 인원 정보』 데이터에서 일별, 시간대별 승·하차 인원을 \n구하기 위해 출근 시간대인  7시~9시의 승·하차 인원을 추출, 

In [74]:
# analysis_format = f"""format"""
# analysis_prompt = f"""당신은 표절 검사 분석 전문가입니다. 표절 검사를 진행한 결과, 특정 비교 문서와 유사하다는 결과가 나왔습니다.
# 세 개의 역 따옴표, 세 개의 따옴표, 세 개의 큰 따옴표로 구분된 각 텍스트들은 검사 문서 중 일부분,비교 문서 중 일부분, 분석 결과 보고서 포맷입니다.
# 분석 결과 보고서 작성해주세요.\n```{checkDoc}```\n\'\'\'{comparisonDoc}\'\'\'\n\"\"\"{f}\"\"\""""

NameError: name 'checkDoc' is not defined

In [ ]:
# You are an expert in plagiarism check analysis. You ran a plagiarism check and found that the document is similar to certain comparison documents.
# The text separated by three inverted quotation marks, three quotation marks, and three double quotation marks is an excerpt from the checked document, an excerpt from the comparison document, and the format of the analysis results report.
# Please create a report of the analysis results.\n```{checkDoc}```\n\'\'\'{comparisonDoc}\'\'\'\'\n\"\"\"{f}\"\"\"

In [80]:
analysis_format = f"""format"""
for doc, score in resList:
    comparisonDoc = doc.page_content
    # checkDoc = document
    checkDoc = "hi"
    analysis_prompt = f"""당신은 표절 검사 분석 전문가입니다. 표절 검사를 진행한 결과, 특정 비교 문서와 유사하다는 결과가 나왔습니다.
세 개의 역 따옴표, 세 개의 따옴표, 세 개의 큰 따옴표로 구분된 각 텍스트들은 검사 문서 중 일부분,비교 문서 중 일부분, 분석 결과 보고서 포맷입니다.
분석 결과 보고서 작성해주세요.\n```{checkDoc}```\n\'\'\'{comparisonDoc}\'\'\'\n\"\"\"{analysis_format}\"\"\""""
    print(len(analysis_prompt))

1173
1173
1173
1187


In [85]:
resList = []
for doc in docs:
    for r in retriever.get_relevant_documents(doc.page_content):
        resList.append((doc, r))

In [103]:
for original, why in resList[:1]:
    # print(original.page_content[:20], why.page_content[:20])
    o = original.page_content[:20]
    w = why.page_content[:20]
    prompt = f"""세 개의 역 따옴표를 구분된 텍스트 2개는 어떠한 부분에서 비슷하다고 유사하다고 할 수 있나요?
    
    ```{o}```
    ```{w}```
    """
    print(prompt)

세 개의 역 따옴표를 구분된 텍스트 2개는 어떠한 부분에서 비슷하다고 유사하다고 할 수 있나요?
    
    ```서울특별시 버스 노선 혼잡도 예측을 ```
    ```서울특별시 버스 노선 혼잡도 예측을 ```
    


## 요약

In [ ]:
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce",
                             verbose = False)

output_summary = chain.run(docs)